In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import qgrid
from glob import glob
import os
import functools
from collections import OrderedDict
from IPython.display import HTML

def header(text, size=100):
    print('*'*size)
    print("{:{fill}{align}{width}}".format("  "+text+"  ", fill='*', align='^', width=size))
    print('*'*size)

In [2]:
saved_path='/nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/verilog_helper/auto_checkers/'
saved_session_name=None

In [3]:
'''
****************************************************************************************************
****************************************  checkers table:  *****************************************
****************************************************************************************************
'''
a=5

In [4]:
default_clocks_table=pd.DataFrame([[False, 'tb.clock.path', 'tb.enable.path', 'tb.clock_freq', 'my clock']], columns=['active_checker','clock_path','checker_enable_net','estimated_freq_out', 'signal_alias'])
default_sleeve_table=pd.DataFrame([[False, 'tb.voltage.path', 'tb.enable.path', 0.4, 0.2, 'dac_vref']], columns=['active_checker','signal_path','checker_enable_net','max_voltage', 'min_voltage','signal_alias'])
default_sleeve_table['sleeve_type']=pd.Categorical(['sleeve'], categories={'sleeve','voltage_source_sleeve', 'frequency_sleeve', 'common_sleeve','current_source_sleeve','resistance_sleeve'})
default_convergence_table = pd.DataFrame([[False, 'tb.code.path', 'tb.enable.path',3, 19, 2, 'tb.convergence_status.path', 'my code']], 
                                           columns=['active_checker','code_path','checker_enable_net','max_change_when_converging', 'stable_time_ps','sleeve_threshold', 'convergance_status', 'signal_alias'])
tables=OrderedDict()
tables['sleeve']      = qgrid.show_grid(default_sleeve_table, show_toolbar=True)
tables['convergence'] = qgrid.show_grid(default_convergence_table, show_toolbar=True)
tables['clock']       = qgrid.show_grid(default_clocks_table, show_toolbar=True)

In [5]:
' generate checkers code tab'
def generate_checkers_func(dummy=None):
    generate_checkers_output.clear_output()
    with generate_checkers_output:
        output_code=list()

        sleeve_checker_code='''checkers.{sleeve_type}(.signal({signal_path}), .checking_condition({checker_enable_net}), .lowThreshold({min_voltage}), .highThreshold({max_voltage}), .netName("{signal_alias}"));'''
        code=tables['sleeve'].get_changed_df().query('active_checker').apply(lambda r: sleeve_checker_code.format(**r), axis=1)
        output_code+=code.values.tolist()
        
        convergence_checker_code='''checkers.converge(.signalMonotonic({code_path}), .en_start_run_checker({checker_enable_net}), .tolerance({max_change_when_converging}), .stable_time_ps({stable_time_ps}), .sleeveThreshold({sleeve_threshold}), .convergence_status({convergence_status}), .netName("{signal_alias}"));'''
        code=tables['convergence'].get_changed_df().query('active_checker').apply(lambda r: convergence_checker_code.format(**r), axis=1)
        output_code+=code.values.tolist()
        
        clock_checker_code='''checkers.clock_frequency(.clock({clock_path}), .clock_freq_ghz({estimated_freq_out}), .checking_condition({checker_enable_net}), .netName("{signal_alias}"));'''
        code=tables['clock'].get_changed_df().query('active_checker').apply(lambda r: clock_checker_code.format(**r), axis=1)
        output_code+=code.values.tolist()
        
        
        if not len(output_code):
            print('no active checker, please update the checkers tables')
        else:
            print('`include "/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/play_ground/checkersPlayground/auto_checkers_gui/checkers.sv";')
            print('module msv_checkers();')
            print('msv_checkers_code checkers();')
            print('initial begin\n\t//$vcdpluson;')
            print("\t"+"\n\t".join(output_code))
            print('end')
            print('endmodule')

run_generate_checkers=widgets.Button( description='generate checkers', disabled=False, button_style='', tooltip='Click me', icon='check')
generate_checkers_output = widgets.Output()

run_generate_checkers.on_click(generate_checkers_func)
generate_checkers_tab=widgets.VBox([run_generate_checkers,generate_checkers_output])

In [6]:
' tables_super_tab '
tables_super_tab = widgets.Tab()
tables_super_tab.children=list(tables.values())
for i, key in enumerate(tables.keys()):
    tables_super_tab.set_title(i, key)

In [7]:
'''
****************************************************************************************************
*******************************************  parse log:  *******************************************
****************************************************************************************************
'''
a=5

In [8]:
' parse log super tab'
def parse_log_func(dummy=None):
    parse_log_output.clear_output()
    with parse_log_output:
        print('TODO parsing log '+log_path.value)
        
        
run_parse_logs=widgets.Button(description='parse logs', disabled=False, button_style='', tooltip='Click me', icon='check' )
log_path=widgets.Text(value='/log/full/path',placeholder='Type something',description='log path:',layout=dict(width='50%'))
parse_log_output = widgets.Output()
run_parse_logs.on_click(parse_log_func)
parse_log_super_tab=widgets.VBox([log_path,run_parse_logs, parse_log_output])

In [9]:
'''
****************************************************************************************************
***************************************  save load session:  ***************************************
****************************************************************************************************
'''
a=5

In [10]:
' save load download session '
def save_session_func(dummy, saved_path, saved_session_name_str, output_widget):
    saved_file=saved_path+saved_session_name_str.value.replace('.xlsx','')+'.xlsx'
    output_widget.clear_output()
    with output_widget:
        print('saving '+saved_file)
    writer = pd.ExcelWriter(saved_file, engine='xlsxwriter')
    for key in tables.keys():
            tables[key].get_changed_df().to_excel(writer, sheet_name=key)
    writer.save()
    !chmod -R 777 $saved_path
    with output_widget:
        print('done saving '+saved_file)
        print(r'win path  \\isamba.iil.intel.com'+saved_file.replace('/','\\'))
        print('you will not see the new file at the result folder until you refresh the browser...')
        
        
def load_session_func(dummy, saved_path, pick_session_str, output_widget, table, picker_widget):
    saved_file=saved_path+pick_session_str.value
    output_widget.clear_output()
    with output_widget:
        print('reading '+saved_file)
        exl=pd.read_excel(saved_file, None)
        for key in table.keys():
            try:
                print('reading sheet '+key)
                table[key].df=exl[key].drop('Unnamed: 0', axis = 1)
            except:
                print('your excel doesnt have %s sheet. leaving last values at this table'%key)
        print('done updating tables')
    refresh_main_output()
        
def download_session_func(dummy, saved_path, pick_session_str, output_widget):
    saved_file=saved_path+pick_session_str.value
    output_widget.clear_output()
    with output_widget:
        print('file path is '+saved_file)
        ref=r'\\isamba.iil.intel.com'+saved_file.replace('/','\\')
        print(r'win path  \\isamba.iil.intel.com'+saved_file.replace('/','\\'))
#         print(ref)
#         display(HTML('''<a href="'''+ref+'''" download="downloaded_file_name" target="_blank"><button type="button">Download</button></a>'''))
        
        
saved_session_name=widgets.Text(value='stam',placeholder='session name',description='session name:',layout=dict(width='50%'))
pick_session=widgets.Select(options=[f.split('/')[-1] for f in glob(saved_path+'/[a-z0-9_]*xlsx')],description='result folder')
save_session_button=widgets.Button(description='save session', button_style='', tooltip='Click me', icon='check')
load_session_button=widgets.Button(description='load session', button_style='', tooltip='Click me', icon='check')
download_session_button=widgets.Button(description='download session', button_style='', tooltip='Click me', icon='check')
save_load_status_output = widgets.Output()

save_session_button.on_click(functools.partial(save_session_func, saved_path=saved_path, saved_session_name_str=saved_session_name, output_widget=save_load_status_output))
load_session_button.on_click(functools.partial(load_session_func, saved_path=saved_path, pick_session_str=pick_session, output_widget=save_load_status_output, table=tables, picker_widget=pick_session))
download_session_button.on_click(functools.partial(download_session_func, saved_path=saved_path, pick_session_str=pick_session, output_widget=save_load_status_output))

save_load_download_tab=widgets.VBox([saved_session_name, pick_session, save_session_button, load_session_button, download_session_button, save_load_status_output])

In [11]:
'''
****************************************************************************************************
****************************************  tabs placments:  *****************************************
****************************************************************************************************
'''
a=5

In [12]:
' handle_checkers_super_tab '
handle_checkers_super_tab = widgets.Tab()
handle_checkers_super_tab.children = [generate_checkers_tab, save_load_download_tab]
handle_checkers_super_tab.set_title(0, 'generate checkers')
handle_checkers_super_tab.set_title(1, 'save load checkers')
# todo move generaet checker and save load checker to different tab and all the checkers tables to the first tab

In [13]:
main_tab = widgets.Tab(layout=dict(width='1200px'))
main_tab.children = [tables_super_tab, handle_checkers_super_tab, parse_log_super_tab]
main_tab.set_title(0, 'checkers tables')
main_tab.set_title(1, 'handle checkers')
main_tab.set_title(2, 'parse log checkers')


main_output= widgets.Output()
def refresh_main_output():
    global main_output
    main_output.clear_output()
    with main_output:
        display(main_tab)
    display(main_output)

refresh_main_output()

Output()

In [14]:
if 0:
    ' i dont have checker for clock...'
    clock_checker_code='''checkers.converge(.signalMonotonic({code_path}), .en_start_run_checker({enble_path}), .tolerance({max_change_when_converging}), .stable_time_ps({stable_time_ps}), .sleeveThreshold({sleeve_threshold}), .convergance_status({convergance_status}), .netName({signal_alias}));'''
    code=tables['clock'].get_changed_df().query('active_checker').apply(lambda r: clock_checker_code.format(**r), axis=1)
    print("\n".join(code.values.tolist()))
    tables['clock'].get_changed_df()
main_tab.children[0].children[1]
tables_super_tab.children=list(tables.values())
# main_tab.children = [tables_super_tab, handle_checkers_super_tab, parse_log_super_tab]